 ## 1. Lectura de datos

 Se comenzará con la lectura del corpus. Para lo cual se utilizarán distintas técnicas de normalización de términos, entre las que se incluye la identificación de palabras compuestas.

In [1]:
!pip install nltk
import nltk

from nltk import download

download('punkt_tab')                           # Tokenización
nltk.download('averaged_perceptron_tagger')     # POS tagging
nltk.download('averaged_perceptron_tagger_eng') # POS tagging
nltk.download('wordnet')                        # WordNet lemmatizer
nltk.download('omw-1.4')                        # WordNet multilingüe

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\marta\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\marta\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\marta\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\marta\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from nltk.data import path
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
import numpy as np

path.append(".")

In [3]:
!pip install contractions
import contractions

In [4]:
import csv
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint
import re
from bs4 import MarkupResemblesLocatorWarning
import warnings

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
import spacy

In [27]:
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
!pip install whoosh
from whoosh.index import create_in
from whoosh.index import open_dir
from whoosh.fields import Schema, TEXT, ID
from whoosh.qparser import QueryParser
from whoosh.query import Term, Or
import os
import shutil

In [28]:
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

In [29]:
palabras_vacias_ingles = stopwords.words('english')

In [30]:
nlp = spacy.load("en_core_web_sm")

In [111]:
def elimina_html(contenido):
    return BeautifulSoup(contenido).get_text()

def elimina_no_alfanumerico(contenido):
    return [re.sub(r'[^\w]', '', palabra)
            for palabra in contenido
            if re.search(r'\w', palabra)]

def expandir_constracciones(contenido):
    return contractions.fix(contenido)

def pasar_a_minuscula(contenido):
    return contenido.lower()

def limpiar_texto(texto):
    texto = re.sub(r'[^a-zA-Z\s]', ' ', texto)  # Reemplaza todo lo que no es letra o espacio con espacio
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

def elimina_palabras_vacias(contenido):
    return [palabra for palabra in contenido if palabra not in palabras_vacias_ingles]

def lematizador(contenido):
    lemmatizer = WordNetLemmatizer()
    pos_tags = pos_tag(contenido)

    resultado = []
    for palabra, tag in pos_tags:
        if tag.startswith('VB'):  # Verbos
            resultado.append(lemmatizer.lemmatize(palabra, pos='v'))  # infinitivo
        else:  # Sustantivos y el resto tal como están
            resultado.append(palabra)

    return resultado

def extraer_noun_chunks(tokens):
    resultados = []
    doc = nlp(" ".join(tokens))

    noun_chunks = [chunk.text.lower().strip() for chunk in doc.noun_chunks if len(chunk.text.split()) <= 3]
    noun_chunks_set = set(noun_chunks)

    i = 0
    while i < len(tokens):
        composed2 = " ".join(tokens[i:i+2]).lower()
 
        if composed2 in noun_chunks_set:
            i += 2 
        else:
            resultados.append(tokens[i].lower())
            i += 1

    return resultados + noun_chunks


In [112]:
def proceso_contenido(texto):
    texto = elimina_html(texto)
    texto = expandir_constracciones(texto)
    texto = pasar_a_minuscula(texto)
    texto = limpiar_texto(texto)                # Limpiar antes de tokenizar
    tokens = word_tokenize(texto)               
    tokens = elimina_no_alfanumerico(tokens)    # Limpiar tokens individuales
    tokens = elimina_palabras_vacias(tokens)
    tokens = lematizador(tokens)
    return tokens

 ## 2. Sistema de recuperación de la información que admite consultas especificadas en texto libre 

 Se procederá con el diseño y la evaluación de un sistema de recuperación de información que admite consultas en texto libre. Este se pondrá a prueba, por un lado, utilizando los documentos del corpus junto con sinónimos de las palabras contenidas en ellos, y por otro lado, sin recurrir a dichos sinónimos.

In [113]:
def lectura_normalizada_corpus():
    df = pd.read_csv("news_corpus.csv", encoding="latin-1", sep=";", quotechar='"')
    resultados = []

    for index, fila in df.iterrows():
        autor = [fila.iloc[0]]
        titulo = fila.iloc[1]
        cuerpo = fila.iloc[2]   

        titulo_proc = proceso_contenido(titulo)
        cuerpo_proc = proceso_contenido(cuerpo)

        # Unir las tres listas en una sola lista combinada
        fila_combinada =  autor + titulo_proc + cuerpo_proc

        contenido_final = extraer_noun_chunks(fila_combinada)
        resultados.append(contenido_final)
    return resultados

def expand_term(term):
    related = set()
    for syn in wn.synsets(term):
        for lemma in syn.lemmas():
            word = lemma.name().replace('_', ' ').lower()
            if word != term:
                related.add(word)
    return related

def expand_corpus_with_synonyms(documento):
    doc_counter = Counter(documento)
    expanded_doc = []
    for word, count in doc_counter.items():
        # Añadimos la palabra original tantas veces como aparece
        expanded_doc.extend([word] * count)
        # Obtenemos sinónimos y también los añadimos con la misma frecuencia
        synonyms = expand_term(word)
        for syn in synonyms:
            expanded_doc.extend([syn] * count)
    return expanded_doc

def lectura_normalizada_corpus_sinonimos():
    df = pd.read_csv("news_corpus.csv", encoding="latin-1", sep=";", quotechar='"')
    resultados = []

    for index, fila in df.iterrows():
        autor = [fila.iloc[0]]
        titulo = fila.iloc[1]
        cuerpo = fila.iloc[2]   

        titulo_proc = proceso_contenido(titulo)
        cuerpo_proc = proceso_contenido(cuerpo)

        # Unir las tres listas en una sola lista combinada
        fila_combinada =  autor + titulo_proc + cuerpo_proc

        contenido_con_palabras_compuestas = extraer_noun_chunks(fila_combinada)
        contenido_final = expand_corpus_with_synonyms(contenido_con_palabras_compuestas)

        resultados.append(contenido_final)
    return resultados

def crear_indice_whoosh(corpus_normalizado):
    if os.path.exists("indice_whoosh"):
        shutil.rmtree("indice_whoosh")  # Elimina índice anterior si existe
    os.mkdir("indice_whoosh")

    schema = Schema(id=ID(stored=True, unique=True), contenido=TEXT(stored=True))
    ix = create_in("indice_whoosh", schema)
    writer = ix.writer()

    for i, doc in enumerate(corpus_normalizado):
        contenido = " ".join(doc)
        writer.add_document(id=str(i), contenido=contenido)
    
    writer.commit()


def buscar_con_whoosh(tokens_query, corpus_normalizado):
    ix = open_dir("indice_whoosh")  # Abrir índice Whoosh previamente creado

    with ix.searcher() as searcher:
        # Crear la consulta OR con todos los términos
        terms = [Term("contenido", token) for token in tokens_query]
        query = Or(terms)

        resultados = searcher.search(query, limit=None)

        # Crear diccionario: clave = doc_id, valor = documento procesado
        docs_encontrados = []
        indices_docs = []
        for hit in resultados:
            doc_id = int(hit['id'])
            docs_encontrados.append(corpus_normalizado[doc_id])
            indices_docs.append(doc_id)

        return docs_encontrados, indices_docs



In [114]:
def lectura_documento(documento):
    documento_procesado = proceso_contenido(documento) 
    contenido_final = extraer_noun_chunks(documento_procesado)
    return contenido_final

def lectura_documento_sinonimos(documento):
    documento_procesado = proceso_contenido(documento) 
    contenido_final = extraer_noun_chunks(documento_procesado)
    resultado = expand_corpus_with_synonyms(contenido_final)
    return resultado

In [115]:
# Mostrar los primeros 3 documentos procesados
def prueba_primeros_3_documentos_procesados(corpus):
    for i, documento in enumerate(corpus[:3]):
        print(f"Documento {i+1}:")
        print(" - Palabras:", documento)
        print()

In [116]:
def tfidf_del_documento(documento_normalizado, vectorizer):
    # Convertir el documento (lista de tokens) a string
    texto = " ".join(documento_normalizado)
    # Transformar usando el vectorizador ya entrenado
    X_doc = vectorizer.transform([texto])  # devuelve matriz sparse 1xN
    return X_doc

In [117]:
def prueba_tfdifs_primeros_3_documentos(lista_diccionarios_tfidfs):
    for idx, d in enumerate(lista_diccionarios_tfidfs[:3]):
        top_terms = sorted(d.items(), key=lambda x: x[1], reverse=True)
        print(f"\n Documento {idx+1}:")
        print("      Palabras añadidas por TF-IDF:")
        for term, score in top_terms:
            print(f"      - {term}: {score:.4f}")

In [118]:
def tfidf_por_corpus(corpus_normalizado):
    # Convertimos el corpus a lista de strings
    texts = [" ".join(doc) for doc in corpus_normalizado]

    # Vectorizador TF-IDF (1 y 2-gramas)
    vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    X = vectorizer.fit_transform(texts)  # Matriz TF-IDF sparse
    terms = vectorizer.get_feature_names_out()

    # Retornamos la matriz y vocabulario (términos)
    return X, terms, vectorizer

In [119]:
def similitud_coseno(tfidf_corpus, tfidf_doc, indices, umbral=0.0):
    # Calcular similitud coseno entre documento y corpus
    similitudes = cosine_similarity(tfidf_doc, tfidf_corpus).flatten()

    # Filtrar documentos que superan el umbral
    indices_filtrados = [i for i, sim in enumerate(similitudes) if sim > umbral]

    # Ordenar índices por similitud descendente
    indices_ordenados = sorted(indices_filtrados, key=lambda i: similitudes[i], reverse=True)

    # Devolver lista de (indice, similitud)
    return [(indices[i], similitudes[i]) for i in indices_ordenados]

def documentos_similares(lista_similitudes):
    lista_similitudes[0]
    for idx, score in lista_similitudes:
        print(f"Documento {idx} tiene similitud: {score:.4f}")
    

In [120]:
#QUITAR SI MAP ABAJO BIEN HECHO
def average_precision(resultados_ordenados, relevantes):
    hits = 0
    sum_precisions = 0
    for i, (idx, _) in enumerate(resultados_ordenados):
        if idx in relevantes:
            hits += 1
            sum_precisions += hits / (i + 1)
    return sum_precisions / len(relevantes) if relevantes else 0

def mean_average_precision(consultas, relevantes_por_consulta, vectorizer, tfidf_corpus):
    all_ap = []
    for i, consulta in enumerate(consultas):
        tfidf_query = tfidf_del_documento(consulta, vectorizer)
        resultados = similitud_coseno(tfidf_corpus, tfidf_query)
        ap = average_precision(resultados, relevantes_por_consulta[i])
        all_ap.append(ap)
        print(f"Consulta {i+1} - AP: {ap:.4f}")
    return sum(all_ap) / len(all_ap)


In [121]:
# Prueba de lectura 
resultados = lectura_normalizada_corpus()
prueba_primeros_3_documentos_procesados(resultados)

Documento 1:
 - Palabras: ['diu', 'revoke', 'mandatory', 'rakshabandhan', 'offices', 'order', 'daman', 'wednesday', 'withdraw', 'circular', 'ask', 'tie', 'rakhis', 'male', 'colleagues', 'order', 'trigger', 'backlash', 'employees', 'rip', 'apart', 'social', 'media', 'union', 'territory', 'administration', 'force', 'retreat', 'within', 'circular', 'make', 'celebrate', 'decide', 'celebrate', 'festival', 'rakshabandhan', 'august', 'connection', 'offices', 'departments', 'shall', 'remain', 'open', 'celebrate', 'festival', 'collectively', 'suitable', 'time', 'wherein', 'shall', 'tie', 'rakhis', 'colleagues', 'order', 'issue', 'august', 'gurpreet', 'singh', 'deputy', 'secretary', 'personnel', 'say', 'ensure', 'one', 'skipped', 'office', 'attendance', 'report', 'send', 'government', 'next', 'even', 'two', 'notifications', 'one', 'mandate', 'celebration', 'rakshabandhan', 'leave', 'withdraw', 'mandate', 'daman', 'day', 'apart', 'circular', 'withdrawn', 'one', 'line', 'order', 'issue', 'late', '

In [150]:
import json

# 2.4. Cargar las consultas TF-IDF
with open("consultas_texto_libre.json", "r", encoding="utf-8") as f:
    consultas = json.load(f)

# Verificación opcional
print(f"Se han cargado {len(consultas)} consultas.")
print("Primera consulta:", consultas[0])
print("Segunda consulta:", consultas[1])

Se han cargado 66 consultas.
Primera consulta: {'query': 'News about Donald Trump', 'documentos': [32, 33, 35, 37, 40, 43]}
Segunda consulta: {'query': 'Information about films, series and shows', 'documentos': [41, 46, 47, 48]}


In [151]:
#Calculo de similitudes para cada consulta sin hacer uso de sinónimos

from sklearn.metrics import average_precision_score
from collections import defaultdict

corpus_normalizado = lectura_normalizada_corpus()
crear_indice_whoosh(corpus_normalizado)  # Crear o recrear índice Whoosh

# Almacenar MAP por consulta
map_scores = []
map_precision= []
map_recall= []

indices_posibles = list(range(149))

for consulta in consultas:
    texto_query = consulta["query"]
    documentos_relevantes = set(consulta["documentos"])  # ya son índices válidos

    # Normalizar la query como si fuera un documento
    lectura = lectura_documento(texto_query)

    # Obtener documentos candidatos mediante Whoosh
    docs_relevantes, indices_docs = buscar_con_whoosh(lectura, corpus_normalizado)

    # Normalizamos el corpus
    tfidf_corpus, terms, vectorizer = tfidf_por_corpus(docs_relevantes)

    # TF-IDF del documento (query)
    tfidf_documento = tfidf_del_documento(lectura, vectorizer)

    # Calcular similitudes
    resultados_similitud = similitud_coseno(tfidf_corpus, tfidf_documento, indices_docs)

    # Convertir a predicciones binarias ordenadas para MAP
    y_true_positive = []
    y_false_positive = []
    y_scores = []
    idxs = []
    no_relevantes = [i for i in indices_posibles if i not in idxs]
    for idx, score in resultados_similitud:
        y_true_positive.append(1 if idx in documentos_relevantes else 0)
        y_false_positive.append(1 if idx not in documentos_relevantes else 0)
        y_scores.append(score)
        idxs.append(idx)
    
    y_true_negative = [1 if i not in documentos_relevantes else 0 for i in no_relevantes]
    y_false_negative = [1 if i in documentos_relevantes else 0 for i in no_relevantes]

    # Calcular Average Precision para esta consulta (si hay al menos un relevante)
    if any(y_true_positive):
        ap = average_precision_score(y_true_positive, y_scores)
        precision = sum(y_true_positive)/(sum(y_true_positive) + sum(y_false_positive))
        recall = sum(y_true_positive)/(sum(y_true_positive) + sum(y_false_negative))
        map_scores.append(ap)
        map_precision.append(precision)
        map_recall.append(recall)

        print(f"Consulta: {texto_query}")
        print(f"Documentos relevantes esperados: {sorted(documentos_relevantes)}")
        print(f"Documentos recuperados ordenados: {[i for i, _ in resultados_similitud]}")
        print(f"Average Precision: {ap:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print("-" * 60)
    else:
        print(f"Consulta: {texto_query}")
        print("No hay documentos relevantes etiquetados para esta consulta. Se omite en MAP.")
        print("-" * 60)


Consulta: News about Donald Trump
Documentos relevantes esperados: [32, 33, 35, 37, 40, 43]
Documentos recuperados ordenados: [134, 32, 35, 37, 40, 33, 96, 43, 16, 100, 9, 19, 20, 110, 11, 107, 138, 53, 2, 60, 54, 36, 133, 58]
Average Precision: 0.7167
Precision: 0.2500
Recall: 0.5000
------------------------------------------------------------
Consulta: Information about films, series and shows
Documentos relevantes esperados: [41, 46, 47, 48]
Documentos recuperados ordenados: [34, 47, 85, 111, 80, 37, 73, 46, 107, 117, 41, 28, 126, 114, 110, 81, 11, 134, 72, 121, 33, 140, 60, 86, 57, 109, 77, 62, 91, 23, 143, 100, 129, 53, 29]
Average Precision: 0.3409
Precision: 0.0857
Recall: 0.4286
------------------------------------------------------------
Consulta: Cases related to racism and hindu culture in offices
Documentos relevantes esperados: [42]
Documentos recuperados ordenados: [0, 42, 61, 82, 103, 50, 138, 59, 45, 71, 20, 7, 114, 19, 113, 25, 36, 115, 144, 135, 4]
Average Precision: 

In [152]:
# Calcular MAP general

print(f"\n======= MÉTRICAS FINALES SIN SINÓNIMOS =======")
if map_scores:
    mean_ap = sum(map_scores) / len(map_scores)
    print(f"Mean Average Precision (MAP): {mean_ap:.4f}")
else:
    print("\nNo se pudo calcular MAP porque ninguna consulta tenía documentos relevantes.")

if map_precision:
    avg_precision = sum(map_precision) / len(map_precision)
    print(f"\n========================")
    print(f"Average Precision: {avg_precision:.4f}")
else:
    print("\nNo se pudo calcular la precisión porque ninguna consulta tenía documentos relevantes.")

if map_recall:
    avg_recall = sum(map_recall) / len(map_recall)
    print(f"\n========================")
    print(f"Average Recall: {avg_recall:.4f}")
else:
    print("\nNo se pudo calcular el recall porque ninguna consulta tenía documentos relevantes.")


======= MÉTRICAS FINALES SIN SINÓNIMOS =======
Mean Average Precision (MAP): 0.8233

Average Precision: 0.0839

Average Recall: 0.4846


In [153]:
#Calculo de similitudes para cada consulta haciendo uso de sinónimos

from sklearn.metrics import average_precision_score
from collections import defaultdict

corpus_normalizado = lectura_normalizada_corpus_sinonimos()
crear_indice_whoosh(corpus_normalizado)  # Crear o recrear índice Whoosh

# Almacenar MAP por consulta
map_scores = []
map_precision= []
map_recall= []

indices_posibles = list(range(149))

for consulta in consultas:
    texto_query = consulta["query"]
    documentos_relevantes = set(consulta["documentos"])  # ya son índices válidos

    # Normalizar la query como si fuera un documento
    lectura = lectura_documento(texto_query)

    # Obtener documentos candidatos mediante Whoosh
    docs_relevantes, indices_docs = buscar_con_whoosh(lectura, corpus_normalizado)

    # Normalizamos el corpus
    tfidf_corpus, terms, vectorizer = tfidf_por_corpus(docs_relevantes)

    # TF-IDF del documento (query)
    tfidf_documento = tfidf_del_documento(lectura, vectorizer)

    # Calcular similitudes
    resultados_similitud = similitud_coseno(tfidf_corpus, tfidf_documento, indices_docs)

    # Convertir a predicciones binarias ordenadas para MAP
    y_true_positive = []
    y_false_positive = []
    y_scores = []
    idxs = []
    no_relevantes = [i for i in indices_posibles if i not in idxs]
    for idx, score in resultados_similitud:
        y_true_positive.append(1 if idx in documentos_relevantes else 0)
        y_false_positive.append(1 if idx not in documentos_relevantes else 0)
        y_scores.append(score)
        idxs.append(idx)
    
    y_true_negative = [1 if i not in documentos_relevantes else 0 for i in no_relevantes]
    y_false_negative = [1 if i in documentos_relevantes else 0 for i in no_relevantes]

    # Calcular Average Precision para esta consulta (si hay al menos un relevante)
    if any(y_true_positive):
        ap = average_precision_score(y_true_positive, y_scores)
        precision = sum(y_true_positive)/(sum(y_true_positive) + sum(y_false_positive))
        recall = sum(y_true_positive)/(sum(y_true_positive) + sum(y_false_negative))
        map_scores.append(ap)
        map_precision.append(precision)
        map_recall.append(recall)

        print(f"Consulta: {texto_query}")
        print(f"Documentos relevantes esperados: {sorted(documentos_relevantes)}")
        print(f"Documentos recuperados ordenados: {[i for i, _ in resultados_similitud]}")
        print(f"Average Precision: {ap:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print("-" * 60)
    else:
        print(f"Consulta: {texto_query}")
        print("No hay documentos relevantes etiquetados para esta consulta. Se omite en MAP.")
        print("-" * 60)


Consulta: News about Donald Trump
Documentos relevantes esperados: [32, 33, 35, 37, 40, 43]
Documentos recuperados ordenados: [32, 35, 134, 37, 96, 40, 33, 43, 16, 69, 100, 135, 9, 65, 84, 107, 7, 11, 133, 60, 110, 20, 95, 2, 118, 36, 19, 8, 117, 46, 53, 80, 138, 70, 54, 86, 141, 3, 139, 57, 129, 59, 6, 13, 12, 128, 81, 79, 76, 42, 74, 147, 0, 115, 5, 18, 55, 51, 82, 68, 25, 62, 39, 24, 56, 4, 47, 146, 58, 29, 119]
Average Precision: 0.8135
Precision: 0.0845
Recall: 0.5000
------------------------------------------------------------
Consulta: Information about films, series and shows
Documentos relevantes esperados: [41, 46, 47, 48]
Documentos recuperados ordenados: [34, 47, 85, 111, 65, 105, 107, 80, 73, 46, 10, 37, 28, 41, 117, 114, 72, 67, 134, 60, 18, 57, 126, 81, 11, 110, 25, 86, 63, 121, 104, 135, 36, 100, 147, 109, 33, 77, 82, 131, 98, 23, 35, 140, 138, 54, 143, 62, 39, 45, 123, 26, 1, 75, 5, 38, 53, 3, 13, 139, 96, 66, 8, 84, 112, 83, 113, 12, 128, 4, 120, 29, 101, 91, 129, 133

In [154]:
# Calcular MAP general

print(f"\n======= MÉTRICAS FINALES CON SINÓNIMOS =======")
if map_scores:
    mean_ap = sum(map_scores) / len(map_scores)
    print(f"Mean Average Precision (MAP): {mean_ap:.4f}")
else:
    print("\nNo se pudo calcular MAP porque ninguna consulta tenía documentos relevantes.")

if map_precision:
    avg_precision = sum(map_precision) / len(map_precision)
    print(f"\n========================")
    print(f"Average Precision: {avg_precision:.4f}")
else:
    print("\nNo se pudo calcular la precisión porque ninguna consulta tenía documentos relevantes.")

if map_recall:
    avg_recall = sum(map_recall) / len(map_recall)
    print(f"\n========================")
    print(f"Average Recall: {avg_recall:.4f}")
else:
    print("\nNo se pudo calcular el recall porque ninguna consulta tenía documentos relevantes.")


======= MÉTRICAS FINALES CON SINÓNIMOS =======
Mean Average Precision (MAP): 0.7781

Average Precision: 0.0224

Average Recall: 0.4915
